In [ ]:
# This is an example playbook. Please do not edit, instead go to File -> Make a Copy

In [ ]:
# setup (always run first)

%pylab inline
import pandas as pd
import json

#return a data frame of the results
def rows_to_df(rows):
    return pd.DataFrame(map(lambda e: e.asDict(), rows))

# return the result as a list of dictionaries
def rows_to_dict(rows):
    return map(lambda e: e.asDict(), rows)

# return an array of the n-th column of the results
def get_column(column_index, rows):
    data = []
    for row in summary:
        data.append(row[column_index])
    return data

# writes the result to a JSON file in the format {"data": [{...}, {...}, ... {...}]}
def write_to_json(file_name, rows):
    with open(file_name, 'w') as outfile:
        json.dump(dict(data = rows_to_dict(rows)), outfile)
    outfile.close()

from pyspark.sql import SQLContext
sqlc = SQLContext(sc)

# this loads the events table from HDFS
rdd = sqlc.read.parquet("/tmp/perpetto-dump-new.parquet")
rdd.registerTempTable("events")

In [ ]:
# statistics (for future reference)

summary = sqlc.sql("SELECT "
                   "entityType, event, targetEntityType, COUNT(*) AS c "
                   "FROM events "
                   "GROUP BY entityType, event, targetEntityType").collect()
rows_to_df(summary)

In [ ]:
# events per user
# histogram

summary = sqlc.sql("SELECT "
                   "count(E1.eventId) "
                   "FROM events E1 "
                   "WHERE E1.entityType = 'user' and E1.targetEntityType = 'item' "
                   "GROUP by E1.entityId").collect()


result_array = get_column(0, summary)
hist(result_array, bins=100, range=(0, 40)) # histogram
sum(result_array)

In [ ]:
# write the results to a file

write_to_json("results.json", summary)